GenAiApp

In [52]:
import os
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
import json
load_dotenv()
model=os.getenv("MODEL_PATH")
base_url=os.getenv("BASE_URL")
model_embedding=os.getenv("EMBEdDING_MODEL")


In [53]:
def get_llm():
    llm=OllamaLLM(model=model,
        base_url=base_url,
        temperature=0.6,
        top_k=30,
        top_p=0.9,
        num_predict=200,
        #format="json",
        #repeat_penalty=1.1,
        #stop=["user"]
    )
    return llm

In [54]:
llm = get_llm()
# llm.invoke("Explain ai",stream=True)

# for chunk in llm.stream("Explain ai"):
#     print(chunk,end="",flush=True)

In [55]:

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.langchain.com/langchain")
docs = loader.load()
docs

[Document(metadata={'source': 'https://www.langchain.com/langchain', 'title': 'LangChain', 'description': 'LangChain is an open source framework with pre-built agent architectures and standard integrations for any model or tool.', 'language': 'en'}, page_content="LangChain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nProducts\n\nLangSmithObservabilityDebug and monitor in-depth tracesEvaluationIterate on prompts and modelsDeploymentShip and scale agents in productionAgent BuilderBetaBuild no-code agentsOpen Source FrameworksLangChainQuick start agents with any model providerLangGraphBuild custom agents with low-level controlDeep AgentsNewUse planning, memory, and sub-agents for complex, long-running tasksLearn\n\nResourcesBlog2026 State of Agent EngineeringCustomer StoriesGuidesChangelogTrust CenterSupportHow-ToLangChain AcademyYouTubeDocumentationCommunityEventsCommunityCommunity ForumSlackDocsCompany\n\nAboutCareersPricingTalk to salesTry LangSmithBuild agents faster, your wayLangCh

In [56]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
textsplitter= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200) 
documets=textsplitter.split_documents(docs)
documets

[Document(metadata={'source': 'https://www.langchain.com/langchain', 'title': 'LangChain', 'description': 'LangChain is an open source framework with pre-built agent architectures and standard integrations for any model or tool.', 'language': 'en'}, page_content='LangChain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nProducts\n\nLangSmithObservabilityDebug and monitor in-depth tracesEvaluationIterate on prompts and modelsDeploymentShip and scale agents in productionAgent BuilderBetaBuild no-code agentsOpen Source FrameworksLangChainQuick start agents with any model providerLangGraphBuild custom agents with low-level controlDeep AgentsNewUse planning, memory, and sub-agents for complex, long-running tasksLearn\n\nResourcesBlog2026 State of Agent EngineeringCustomer StoriesGuidesChangelogTrust CenterSupportHow-ToLangChain AcademyYouTubeDocumentationCommunityEventsCommunityCommunity ForumSlackDocsCompany\n\nAboutCareersPricingTalk to salesTry LangSmithBuild agents faster, your wayLangCh

In [57]:
from langchain_ollama import OllamaEmbeddings   
embeddings=OllamaEmbeddings(model=model_embedding,base_url=base_url)

In [58]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents=documets,embedding=embeddings)
result=vectorstoredb.similarity_search_with_score("What is langchain?",k=10)


In [59]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_template("""
Use the context to answer:

{context}

Question: {question}
""")

parser = StrOutputParser()

In [60]:
retriever=vectorstoredb.as_retriever()
docss= retriever.invoke("What is langchain?")

In [61]:
def format_docs(docss):
    return "\n\n".join(d.page_content for d in docss)


In [62]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | parser
)


In [63]:
#rag_chain.invoke("What is LangChain?")
for chunk in rag_chain.invoke("What is LangChain?"):
     print(chunk,end="",flush=True)

LangChain is an open-source framework that allows developers to build agents quickly by providing pre-built agent architectures and integrations with various models, tools, and databases. It integrates seamlessly with LangSmith, a platform for agent observability, evaluation, and deployment.